Import libraries

In [18]:
import pandas as pd
import geocoder

In [2]:
df = pd.read_csv('data/api_data_tableau.csv')

In [3]:
df.head()

,id,url,launch_library_id,slug,name,net,window_end,window_start,mission,mission_type,pad,location,landing,landing_success,launcher,orbit,image,status.id,status.name
0,c183a50f-fcc0-4cdc-8fa1-750d67ca5130,http://spacelaunchnow.me/api/3.3.0/launch/c183...,2101.0,https://spacelaunchnow.me/launch/electron-in-f...,Electron | In Focus,2020-10-21T21:27:00Z,2020-10-21T22:03:00Z,2020-10-21T21:14:00Z,In Focus,Dedicated Rideshare,Rocket Lab Launch Complex 1A,"Onenui Station, Mahia Peninsula, New Zealand",NaN,NaN,NaN,SSO,https://spacelaunchnow-prod-east.nyc3.cdn.digi...,1,Go
1,47c91a03-2e98-42c9-8751-d8fb36c89c99,http://spacelaunchnow.me/api/3.3.0/launch/47c9...,2107.0,https://spacelaunchnow.me/launch/falcon-9-bloc...,Falcon 9 Block 5 | Starlink 14,2020-10-22T16:14:00Z,2020-10-22T16:14:00Z,2020-10-22T16:14:00Z,Starlink 14,Communications,Space Launch Complex 40,"Cape Canaveral, FL, USA",JRTI,0.0,B1060,LEO,https://spacelaunchnow-prod-east.nyc3.cdn.digi...,1,Go
2,1917999c-4d64-41a7-8a7b-9202c54ca2ef,http://spacelaunchnow.me/api/3.3.0/launch/1917...,1680.0,https://spacelaunchnow.me/launch/soyuz-21bfreg...,Soyuz 2.1b/Fregat | Glonass-K1,2020-10-25T19:08:42Z,2020-10-25T19:08:42Z,2020-10-25T19:08:42Z,Glonass-K1,Navigation,43/3 (43L),"Plesetsk Cosmodrome, Russian Federation",NaN,NaN,NaN,NaN,https://spacelaunchnow-prod-east.nyc3.cdn.digi...,1,Go
3,504ba71a-bf09-48f1-9346-48c3d79e580b,http://spacelaunchnow.me/api/3.3.0/launch/504b...,NaN,https://spacelaunchnow.me/launch/momo-flight-7,MOMO | Flight 7,2020-10-31T00:00:00Z,2020-10-31T00:00:00Z,2020-10-31T00:00:00Z,Flight 7,Suborbital,"Taiki-cho, Hokkaido, Japan","Taiki-cho, Hokkaido, Japan",NaN,NaN,NaN,Sub Orbital,https://spacelaunchnow-prod-east.nyc3.cdn.digi...,2,TBD
4,528b72ff-e47e-46a3-b7ad-23b2ffcec2f2,http://spacelaunchnow.me/api/3.3.0/launch/528b...,2103.0,https://spacelaunchnow.me/launch/falcon-9-bloc...,Falcon 9 Block 5 | NROL-108,2020-10-31T00:00:00Z,2020-10-31T00:00:00Z,2020-10-31T00:00:00Z,NROL-108,Government/Top Secret,Space Launch Complex 40,"Cape Canaveral, FL, USA",NaN,NaN,NaN,NaN,https://spacelaunchnow-prod-east.nyc3.cdn.digi...,2,TBD


In [4]:
df.isnull().sum()

id                      0
url                     0
launch_library_id      34
slug                    0
name                    0
net                     0
window_end              0
window_start            0
mission               689
mission_type          717
pad                     0
location                0
landing              1747
landing_success      1721
launcher             1721
orbit                 780
image                 189
status.id               0
status.name             0
dtype: int64

In [5]:
df['location'].unique()

array(['Onenui Station, Mahia Peninsula, New Zealand',
       'Cape Canaveral, FL, USA',
       'Plesetsk Cosmodrome, Russian Federation',
       'Taiki-cho, Hokkaido, Japan', 'Kourou, French Guiana',
       'Sriharikota, Republic of India',
       "Taiyuan, People's Republic of China", 'Vandenberg AFB, CA, USA',
       'Kennedy Space Center, FL, USA', 'Wallops Island, Virginia, USA',
       "Wenchang Satellite Launch Center, People's Republic of China",
       'Tanegashima, Japan', "Jiuquan, People's Republic of China",
       'Air launch to orbit', 'Unknown Location',
       'Vostochny Cosmodrome, Siberia, Russian Federation',
       'Baikonur Cosmodrome, Republic of Kazakhstan',
       "Xichang Satellite Launch Center, People's Republic of China",
       'Uchinoura Space Center, Japan', 'Air launch to Suborbital flight',
       'Ronald Reagan Ballistic Missile Defense Test Site, Kwajalein Atoll, Marshall Islands',
       'Sea Launch', 'Dombarovskiy, Russian Federation',
       'Palm

In [14]:
# Empty list to store the location which its latitude and its longitude
locations = []

# Loop over the unique locations
for location in df['location'].unique():
    name=location.split(', ')
    
    # Check if the word cosmodrome is inside the name, if so, remove it
    if 'Cosmodrome' in name[0]:
        name[0] = name[0].split(' Cosmodrome')[0]
    # exception with Wenchang satellite launch center, otherwise it cannot find it
    elif 'Wenchang' in name[0]:
        name[0] = 'Wenchang'
    # Check if Palmachim Airbase and remove Airbase word
    elif 'Palmachim' in name[0]:
        name[0] = 'Palmachim'
    elif 'SpaceX Space Launch Facility' in name[0]:
        name[0] = 'Texas'
    elif 'Shahrud Missile Test Site' in name[0]:
        name[0] = 'Iran'
    
    # Check if the location is air launch, then skip it
    if 'Air launch' in name[0]:
        locations.append([location, 0, 0])
        continue
    g = geocoder.osm(name[0])

    locations.append([location, g.json['lat'], g.json['lng']])

In [17]:
# Save the location as csv
locations = pd.DataFrame(locations, columns=['Location', 'Lat', 'Lon'])
locations.to_csv('data/locations.csv', index=False)